# Step 2: Retrieve Documents

This notebook tests the document retrieval node. It loads a pre-built vector store and uses it to find document chunks that are semantically similar to a user's question.

In [1]:
import os
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# Add the project root to the Python path
import sys
sys.path.append('..')

**Important:** Before running this notebook, you must first run the main Streamlit app (`streamlit run main.py`) and click the **"Build Vector Store"** button to create the `vector_store` index.

In [2]:
# --- Configuration ---
MODEL_NAME = "llama3"
VECTOR_STORE_PATH = "../vector_store"

embeddings = OllamaEmbeddings(model=MODEL_NAME)

# --- Load Vector Store Directly ---
faiss_path = os.path.join(VECTOR_STORE_PATH, "index.faiss")
pkl_path = os.path.join(VECTOR_STORE_PATH, "index.pkl")

if os.path.exists(faiss_path) and os.path.exists(pkl_path) and os.path.getsize(faiss_path) > 0:
    print(f"Loading vector store from {VECTOR_STORE_PATH}...")
    vector_store = FAISS.load_local(
        VECTOR_STORE_PATH, 
        embeddings, 
        allow_dangerous_deserialization=True
    )
    retriever = vector_store.as_retriever()
    print("Retriever is ready.")
else:
    raise FileNotFoundError(
        f"Vector store not found or is corrupted at '{VECTOR_STORE_PATH}'. "
        "Please run the main app and click 'Build Vector Store' first."
    )

Loading vector store from ../vector_store...
Retriever is ready.


/var/folders/90/p8kgtvm572d51cr90k19v_580000gn/T/ipykernel_62714/821081663.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=MODEL_NAME)


In [3]:
def retrieve_documents(state: dict, retriever) -> dict:
    """
    Retrieves documents from the vector store.
    This function is defined locally for experimentation.
    """
    print("---RETRIEVING DOCUMENTS---")
    question = state['question']
    documents = retriever.invoke(question)
    return {"documents": documents}

In [4]:
question = "What are the requirements for CEMS data quality?"
state = {"question": question}

result = retrieve_documents(state, retriever)

print(f"Found {len(result['documents'])} relevant documents:")
for i, doc in enumerate(result['documents']):
    print(f"--- Document {i+1} ---")
    print(f"Source: {doc.metadata.get('source')}")
    print(doc.page_content)
    print()

---RETRIEVING DOCUMENTS---
Found 4 relevant documents:
--- Document 1 ---
Source: documents/ghg-emission-factors-hub-2025.pdf
All CO 2 emission factors assume that 100 percent of the carbon content of the fuel is oxidized to CO 2, as is recommended by the Intergovernmental Panel on Climate Change (IPCC). 
The CH 4 and N 2O emission factors provided represent emissions in terms of fuel type and by end-use sector (i.e., residential, commercial, industrial, electricity generation).
Please see the Center for Corporate Climate Leadership's guidance document on Direct Emissions from Stationary Combustion Sources for guidance on how to include biomass fuels in an organization's greenhouse gas inventory.
https://www.epa.gov/sites/default/files/2020-12/documents/stationaryemissions.pdf
The factors represented in the table above represent combustion emissions only and do not represent upstream emissions.Emission Factors for Greenhouse Gas Inventories

--- Document 2 ---
Source: documents/ghg-emi